In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import tensorflow as tf

In [ ]:
# 분노 불러오기

sen = pd.read_csv('분노.csv')
sen.columns = ['index','a']     # 컬럼명 정리
sen = sen.loc[sen['a'].notnull(),:]   # null 처리
only_word = list(sen['a'].str.split())  # 리스트형태로 만들기
rage = np.repeat('2',len(only_word))


# 긍정 불러오기

sen2 = pd.read_csv('긍정.csv')
sen2.columns = ['index','a']    # 컬럼명 정리
sen2 = sen2.loc[sen2['a'].notnull(),:]   # null 처리
only_word2 = list(sen2['a'].str.split())    # 리스트형태로 만들기
positive = np.repeat('1',len(only_word2))


# 중립 불러오기 

sen3 = pd.read_csv('중립.csv')
sen3.columns = ['index','a']    # 컬럼명 정리
sen3 = sen3.loc[sen3['a'].notnull(),:]   # null 처리
only_word3 = list(sen3['a'].str.split())    # 리스트형태로 만들기
neutrality = np.repeat('0',len(only_word3))

In [ ]:
# 감정 별 train, test 분리(y는 필요하지 않음)

df1 = pd.DataFrame({'text':only_word, 'tag' : rage})
df2 = pd.DataFrame({'text':only_word2, 'tag' : positive})
df3 = pd.DataFrame({'text':only_word3, 'tag' : neutrality})

train_x1, test_x1, train_y1, test_y1 = train_test_split(df1, df1, random_state = 0, train_size = 0.8) 
train_x2, test_x2, train_y2, test_y2 = train_test_split(df2, df2, random_state = 0, train_size = 0.8) 
train_x3, test_x3, train_y3, test_y3 = train_test_split(df3, df3, random_state = 0, train_size = 0.8) 

In [ ]:
# 모든 감정 합치기 
df1 = pd.concat([train_x1, train_x2, train_x3, train_x2])
df2 = pd.concat([test_x1, test_x2, test_x3])

In [ ]:
# train, test 개수 확인
print(df1.shape) # (25756, 2)
print(df2.shape) # (8587, 2)

(28978, 2)
(6869, 2)


In [ ]:
# 모든 단어 한개의 리스트 안에 넣기
# tokens = [t for d in s1 for t in d]
# sum(s1,[])로 대체 가능
tokens1 = sum(df1['text'],[]) 
tokens2 = sum(df2['text'],[])

In [ ]:
text = nltk.Text(tokens1, name='NMSC')
text2 = nltk.Text(tokens2, name='NMSC')

In [ ]:
print(len(set(text.tokens))) # 22377, 중복 제거 개수
print(len(set(text2.tokens))) # 13392

23042
11932


In [ ]:
print(text.vocab().most_common(10)) # 가장 많이 쓰인 것

print(text.vocab().most_common()[:-20:-1]) # 가장 적게 쓰인 것 

[('이', 6193), ('하', 5980), ('안', 3718), ('다', 3330), ('없', 2825), ('있', 2761), ('중국', 2681), ('코로나', 2589), ('나라', 2440), ('사람', 2255)]
[('지으면', 1), ('광휘', 1), ('거비', 1), ('차이나타운', 1), ('꾸며낸', 1), ('면세점', 1), ('내막', 1), ('꿍꿍', 1), ('활기차', 1), ('정상인', 1), ('워낙에', 1), ('바려', 1), ('내려보', 1), ('법칙', 1), ('혜인', 1), ('커리어', 1), ('삐리', 1), ('경창', 1), ('역이용', 1)]


In [ ]:
selected_words= [f[0] for f in text.vocab().most_common(5000)] # 상위 5000개 단어 선정

In [ ]:
# BoW를 위한 함수 생성
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [ ]:
dtm_train = [term_frequency(d) for d in df1['text']] # train 데이터들의 BoW
dtm_test = [term_frequency(d) for d in df2['text']] # test 데이터들의 BoW

In [ ]:
# 모델 입력을 위해 array 형태로 변경
train_x = np.array(dtm_train)
test_x = np.array(dtm_test)

In [ ]:
# x 사이즈 확인
print(train_x.shape) # (25756, 5000)
print(test_x.shape) # (8587, 5000)

(28978, 5000)
(6869, 5000)


In [ ]:
# 라벨 인코딩
#from sklearn.preprocessing import LabelEncoder
#encoder = LabelEncoder()
#encoder.fit(df1['tag'])
#encoder.transform(df1['tag'])
#위에서 이미 '0','1','2' 형태로 tag했기 때문에 사용하지는 않음

In [ ]:
# 원 핫 인코딩
labels_train = np.array(df1['tag']).reshape(-1,1)
labels_test = np.array(df2['tag']).reshape(-1,1)
onehotencoder = OneHotEncoder()
onehotencoder.fit(labels_train)
train_y = onehotencoder.transform(labels_train).toarray()
test_y = onehotencoder.transform(labels_test).toarray()

In [ ]:
# keras 모델 생성

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(5000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizers.RMSprop(lr=0.001), 
             loss='categorical_crossentropy',
             metrics=[metrics.categorical_accuracy])

# 감정이 중립, 긍정, 분노이기 때문에 binary가 아닌 categorical 사용

model.fit(train_x, train_y, epochs=10, batch_size=512)

Train on 28978 samples
Epoch 1/10
28978/28978 [==============================] - 16s 564us/sample - loss: 0.8012 - categorical_accuracy: 0.6397
Epoch 2/10
28978/28978 [==============================] - 2s 67us/sample - loss: 0.5637 - categorical_accuracy: 0.7522
Epoch 3/10
28978/28978 [==============================] - 2s 68us/sample - loss: 0.4849 - categorical_accuracy: 0.7865
Epoch 4/10
28978/28978 [==============================] - 2s 70us/sample - loss: 0.4326 - categorical_accuracy: 0.8142
Epoch 5/10
28978/28978 [==============================] - 2s 77us/sample - loss: 0.3887 - categorical_accuracy: 0.8344
Epoch 6/10
28978/28978 [==============================] - 2s 75us/sample - loss: 0.3481 - categorical_accuracy: 0.8553
Epoch 7/10
28978/28978 [==============================] - 2s 76us/sample - loss: 0.3075 - categorical_accuracy: 0.8769
Epoch 8/10
28978/28978 [==============================] - 2s 83us/sample - loss: 0.2676 - categorical_accuracy: 0.8962
Epoch 9/10
28978/28978 

In [ ]:
# test 결과 확인
results = model.evaluate(test_x, test_y)


6869/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [ ]:
print(results)

[1.0481965717902817, 0.69981074]
